# For read file

In [92]:
%pip install kafka-python-ng
%pip install --upgrade avro==1.10.0 kafka-python
%pip install selenium
%pip install chardet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 5.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [96]:
import hashlib
import io
import json
import os
import zipfile
import pandas as pd
import chardet

from avro import schema
from avro.io import DatumWriter, BinaryEncoder
from kafka import KafkaConsumer, KafkaProducer

import re
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

import pprint  # Import pprint for pretty printing

In [13]:
years = range(2018, 2024)  # Years from 2018 to 2023
for year in years:
    # Construct the URL
    url = f"https://github.com/nnatchy/DSDE_Project/raw/main/{year}_test.zip"
    # Construct the wget command to download and rename the file directly
    !wget {url} -O {year}.zip

--2024-05-07 02:41:55--  https://github.com/nnatchy/DSDE_Project/raw/main/2018_test.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nnatchy/DSDE_Project/main/2018_test.zip [following]
--2024-05-07 02:41:56--  https://raw.githubusercontent.com/nnatchy/DSDE_Project/main/2018_test.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8001::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6913340 (6.6M) [application/zip]
Saving to: ‘2018.zip’

2018.zip            100%[===================>]   6.59M  --.-KB/s    in 0.1s    

2024-05-07 02:41:57 (46.1 MB/s) - ‘2018.zip’ saved [6913340/6913340]

--2024-05-07 02:41:57--  https://github.

In [7]:
# use this function to print the json with the indent=4
def jprint(data):
  print(json.dumps(data,indent=4))

In [4]:
def process_files(year_range, base_path):
    batch_years = []
    for year in year_range:
        # Determine the directory pattern based on the year
        year_directory = f"{base_path}/{year}/" if year == 2018 else f"{base_path}/{year}/"

        # Ensure the year directory exists and create if not
        os.makedirs(year_directory, exist_ok=True)

        # Check for zip files in the base path and unzip them to the year directory
        zip_file_path = os.path.join(base_path, f'{year}.zip')
        if os.path.exists(zip_file_path):
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(year_directory)
                print(f"Extracted {zip_file_path} to {year_directory}")

by = process_files(range(2018, 2024), ".")

Extracted ./2018.zip to ./2018/
Extracted ./2019.zip to ./2019/
Extracted ./2020.zip to ./2020/
Extracted ./2021.zip to ./2021/
Extracted ./2022.zip to ./2022/
Extracted ./2023.zip to ./2023/


In [128]:
# with open("./2018/2018/201800010", 'r') as file:
#     json_data = json.load(file)
# abstracts_info = json_data.get("abstracts-retrieval-response", {})

# Function for processing data

In [24]:
# function to find how many author are in this paper
def num_author_group(head_data):
  num = 0
  for author in head_data:
    if int(author['seq']) > num:
      num = int(author['seq'])

  return num

In [76]:
def process_head_data(bibrecord_data):
    # process author
    author_groups = bibrecord_data["head"]["author-group"]
    if isinstance(author_groups, dict):
        author_groups = [author_groups]  
    
    head_output_data = {"author_groups": []}

    for author in author_groups:
        affi = author.get("affiliation", {})
        org = affi.get("organization", [])
        if isinstance(org, dict):  
            organization_names = [org["$"]]
        elif isinstance(org, list):  
            organization_names = [o["$"] for o in org]
        else:
            organization_names = []  
    
        affi_info = {
            "affiliation_id": affi.get("@afid", ""),
            "dpt_id": affi.get("@dptid", ""),
            "country": affi.get("country", ""),
            "organization": organization_names  
        }
    
        authors = author.get("author", [])
        if isinstance(authors, dict):  
            authors = [authors]
    
        for person in authors:
            author_info = {
                "indexed-name": person.get("preferred-name", {}).get("ce:indexed-name", ""),
                "seq": person.get("@seq", ""),
                "auid": person.get("@auid", ""),
                "affiliation": affi_info  # Include affiliation info
            }
            head_output_data["author_groups"].append(author_info)
    head_output_data["enhancement"] = []

    # process classifiaction group
    if "enhancement" in bibrecord_data["head"] and "classificationgroup" in bibrecord_data["head"]["enhancement"]:
        classifications = bibrecord_data["head"]["enhancement"]["classificationgroup"].get("classifications", [])

        if isinstance(classifications, dict):
            classifications = [classifications] 
    
        for classification in classifications:
            if classification["@type"] == "SUBJABBR":  
                if isinstance(classification.get("classification"), (list, dict)):
                    if isinstance(classification["classification"], list):
                        all_classifications = [item.get("$", item) if isinstance(item, dict) else item for item in classification["classification"]]
                    else:  
                        all_classifications = [classification["classification"].get("$", classification["classification"])]
                else:  
                    all_classifications = [classification["classification"]]
    
                class_info = {
                    "type": classification["@type"],
                    "classifications": all_classifications
                }
                head_output_data["enhancement"].append(class_info)

    
    return head_output_data


In [61]:
import requests

def get_citation_count(DOI):
    
    API_KEY = "8a4d62cf82cf68382f5f55a658354336"
    BASE_URL = "http://api.elsevier.com/content/search/scopus"
    
    headers = {
        "X-ELS-APIKey": API_KEY
    }
    
    query_url = f"{BASE_URL}?query=DOI({DOI})&field=citedby-count"
    response = requests.get(query_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        cited_by_count = data["search-results"]["entry"][0]["citedby-count"]
        # print(f"The document with DOI {DOI} has been cited by {cited_by_count} times in Scopus.")
    else:
        print("Error:", response.status_code)

    return cited_by_count

In [64]:
def process_tail_data(bibrecord_data):
    # Ensure 'tail' is a dictionary before proceeding
    tail_info = bibrecord_data.get("tail", {})

    bibliography_output = {
        "refcount": "",
        "references": []
    }

    if not tail_info:  # Check if tail_info is empty or None
        return bibliography_output

    bibliography = tail_info.get("bibliography", {})
    bibliography_output["refcount"] = bibliography.get("@refcount", "0")

    # Handle references
    references = bibliography.get("reference", [])
    if not isinstance(references, list):
        references = [references] if references else []

    for ref in references:
        reference_info = {
            "id": ref.get("@id", ""),
            "ref_fulltext": ref.get("ref-fulltext", ""),
            "ref_text": ref.get("ce:source-text", ""),
            "ref_info": {},
            "ref_authors": [],
            "ref_authors_count": "",
            "ref_collab": []
        }

        ref_info = ref.get("ref-info", {})
        reference_info["ref_info"] = {
            "ref_publicationyear": ref_info.get("ref-publicationyear", {}).get("@first", ""),
            "ref_title": ref_info.get("ref-title", {}).get("ref-titletext", "Title Not Available"),
            "ref_sourcetitle": ref_info.get("ref-sourcetitle", "")
        }

        # Process authors and collaborations
        ref_authors = ref_info.get("ref-authors", {})
        authors = ref_authors.get("author", [])
        if not isinstance(authors, list):
            authors = [authors] if authors else []
        reference_info["ref_authors"] = [author.get("ce:indexed-name", "") for author in authors]
        reference_info["ref_authors_count"] = len(reference_info["ref_authors"])

        collaborations = ref_authors.get("collaboration", [])
        if not isinstance(collaborations, list):
            collaborations = [collaborations] if collaborations else []
        reference_info["ref_collab"] = [{"collaboration_name": collab.get("ce:text", "")} for collab in collaborations]

        bibliography_output["references"].append(reference_info)

    return bibliography_output


# Function for get the required Feature

In [125]:
def get_all_feature(data, file_name):
    output_data = {
        "id": str(file_name),
        "source": 1,
        "coAuthorship": 0,
        "citationCount": 0,
        "refCount": 0,
        "Class": []
    }

    bibrecord_data = data['item']['bibrecord']
    
    # find num authorship
    head_data = process_head_data(bibrecord_data)
    output_data['coAuthorship'] = num_author_group(head_data['author_groups'])

    # find citation count
    item_info = bibrecord_data['item-info']
    if "ce:doi" not in item_info['itemidlist']:
        output_data['citationCount'] = None
    else :
        doi = item_info['itemidlist']['ce:doi']
        output_data['citationCount'] = get_citation_count(doi)

    # find the references count
    tail_data = process_tail_data(bibrecord_data)
    output_data['refCount'] = tail_data['refcount']

    # get the class
    enhancement_data = head_data['enhancement']
    for item in enhancement_data:
        if item['type'] == 'SUBJABBR':
            for subject in item['classifications']:
                output_data['Class'].append(subject)
    
    return output_data

# Kafka

In [ ]:
from kafka import KafkaProducer
import time
import json

producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda x: json.dumps(x).encode('utf-8'),
    request_timeout_ms=60000,  # Increase timeout to 60 seconds
    retry_backoff_ms=500,  # Increase backoff time between retries
)

def process_directory(base_path):
    for year in range(2018, 2019):  # Assuming years 2018 to 2023
        year_path = os.path.join(base_path, str(year), f'{year}')
        if os.path.isdir(year_path):
            for file_name in os.listdir(year_path):
                file_path = os.path.join(year_path, file_name)
                # Detect file encoding
                with open(file_path, 'rb') as f:  # open in binary mode
                    raw_data = f.read()
                    result = chardet.detect(raw_data)
                    encoding = result['encoding']

                # Read the file with detected encoding
                with open(file_path, 'r', encoding=encoding) as file:
                    json_data = json.load(file)
                    abstracts_info = json_data.get("abstracts-retrieval-response", {})
                    data = get_all_feature(abstracts_info, file_name)
                    try:
                        producer.send('test-topic', value=data).get(timeout=30)  # Wait up to 30 seconds
                        print(f"Sent: {data}")
                    except Exception as e:
                        print(f"Failed to send message: {str(e)}", file_name)
                    time.sleep(1)


process_directory('./')

producer.flush()